# CubeSharp Comprehensive Tutorial

Welcome to the comprehensive CubeSharp tutorial! This notebook will guide you through all aspects of building and querying multi-dimensional data cubes using the CubeSharp library.

## What is CubeSharp?

CubeSharp is a lightweight .NET library for building in-memory data cubes. It provides a systematic approach to multi-dimensional data analysis and aggregation, enabling developers to create flexible, testable, and maintainable reporting solutions.

## Why Use CubeSharp?

Traditional approaches to multi-dimensional reporting present several challenges:
- **SQL Limitations**: Limited refactoring and testing capabilities
- **Ad-hoc LINQ Solutions**: Code duplication and maintenance difficulties
- **CubeSharp Solution**: Systematizes multi-factor reports with structured APIs


## Setup

First, let's add the CubeSharp package and necessary using statements:

In [1]:
#r "nuget: CubeSharp, 99.0.0.3-alpha.nupkg"

using CubeSharp;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

display("CubeSharp loaded successfully!");

Error: C:\Users\Gennadii_Saltyshchak\.packagemanagement\nuget\Projects\36432--1353d17b-e1ea-4b76-bf5d-795d4764dadf\Project.fsproj : error NU1102: Unable to find package CubeSharp with version (>= 99.0.0.3-alpha.nupkg)
C:\Users\Gennadii_Saltyshchak\.packagemanagement\nuget\Projects\36432--1353d17b-e1ea-4b76-bf5d-795d4764dadf\Project.fsproj : error NU1102:   - Found 3 version(s) in local [ Nearest version: 99.0.0.3-alpha ]
C:\Users\Gennadii_Saltyshchak\.packagemanagement\nuget\Projects\36432--1353d17b-e1ea-4b76-bf5d-795d4764dadf\Project.fsproj : error NU1102:   - Found 0 version(s) in C:\Program Files\dotnet\library-packs
C:\Users\Gennadii_Saltyshchak\.packagemanagement\nuget\Projects\36432--1353d17b-e1ea-4b76-bf5d-795d4764dadf\Project.fsproj : error NU1102:   - Found 0 version(s) in C:\Program Files\dotnet\sdk\9.0.304\FSharp\library-packs
C:\Users\Gennadii_Saltyshchak\.packagemanagement\nuget\Projects\36432--1353d17b-e1ea-4b76-bf5d-795d4764dadf\Project.fsproj : error NU1102:   - Fou

In [2]:
using System.Collections.Generic;
using System.Reflection;
using Microsoft.Data.Analysis;

public static DataFrame ToDataFrame<T>(this IEnumerable<T> source)
{
    var props = typeof(T).GetProperties(BindingFlags.Public | BindingFlags.Instance);
    var columns = props
        .Select(p => CreateColumn(p.Name, p.PropertyType))
        .ToList();
    var result = new DataFrame(columns);
    foreach (var item in source)
    {
        result.Append(props.ToDictionary(p => p.Name, p => p.GetValue(item)), true);
    }
    return result;

    static DataFrameColumn CreateColumn(string name, Type type) =>
        type switch
        {
            var t when t == typeof(bool) => new BooleanDataFrameColumn(name),
            var t when t == typeof(int) => new Int32DataFrameColumn(name),
            var t when t == typeof(decimal) => new DecimalDataFrameColumn(name),
            var t when t == typeof(double) => new DoubleDataFrameColumn(name),
            var t when t == typeof(DateTime) => new DateTimeDataFrameColumn(name),
            var t when t == typeof(string) => new StringDataFrameColumn(name),
            _ => throw new NotSupportedException($"Type {type} not supported")
        };
}

display("Utility function ToDataFrame<T> loaded successfully!");

Error: (3,17): error CS0234: The type or namespace name 'Data' does not exist in the namespace 'Microsoft' (are you missing an assembly reference?)
(5,15): error CS0246: The type or namespace name 'DataFrame' could not be found (are you missing a using directive or an assembly reference?)
(11,22): error CS0246: The type or namespace name 'DataFrame' could not be found (are you missing a using directive or an assembly reference?)
(21,49): error CS0246: The type or namespace name 'BooleanDataFrameColumn' could not be found (are you missing a using directive or an assembly reference?)
(22,48): error CS0246: The type or namespace name 'Int32DataFrameColumn' could not be found (are you missing a using directive or an assembly reference?)
(23,52): error CS0246: The type or namespace name 'DecimalDataFrameColumn' could not be found (are you missing a using directive or an assembly reference?)
(24,51): error CS0246: The type or namespace name 'DoubleDataFrameColumn' could not be found (a

## Chapter 1: Getting Started - Basic Example

Let's start with a practical example using order data. We'll create a report with customers as rows, years as columns, and total quantity in each cell.

In [3]:
// Sample order data
record Order {
    public DateTime OrderDate { get; init; }
    public string Product { get; init; }
    public int EmployeeId { get; init; }
    public string CustomerId { get; init; }
    public decimal Quantity { get; init; }
}

var orders = new Order[] {
    new Order
    {
        OrderDate = new DateTime(2007, 08, 02),
        Product = "Laptop",
        EmployeeId = 3,
        CustomerId = "A",
        Quantity = 10m,
    },
    new Order
    {
        OrderDate = new DateTime(2007, 12, 24),
        Product = "Mouse",
        EmployeeId = 4,
        CustomerId = "B",
        Quantity = 12m,
    },
    new Order
    {
        OrderDate = new DateTime(2008, 03, 15),
        Product = "Keyboard",
        EmployeeId = 3,
        CustomerId = "A",
        Quantity = 30m,
    },
    new Order{
        OrderDate = new DateTime(2008, 06, 10),
        Product = "Monitor",
        EmployeeId = 5,
        CustomerId = "C",
        Quantity = 14m,
    },
    new Order
    {
        OrderDate = new DateTime(2009, 01, 20),
        Product = "Tablet",
        EmployeeId = 4,
        CustomerId = "D",
        Quantity = 60m,
    }
};

display($"Sample data loaded: {orders.Length} orders");
display("Order details:");
display(orders.ToDataFrame());

Error: (54,16): error CS1061: 'Order[]' does not contain a definition for 'ToDataFrame' and no accessible extension method 'ToDataFrame' accepting a first argument of type 'Order[]' could be found (are you missing a using directive or an assembly reference?)

### Step 1: Define How to Aggregate Data

An aggregation definition specifies:
1. **Value Selector**: What value to extract from each record
2. **Aggregation Function**: How to combine multiple values
3. **Seed Value**: Initial value and default for empty cells

In [4]:
// Define how to aggregate data (sum of quantities)
var aggregationDefinition = AggregationDefinition.CreateForCollection(
    orders,
    order => order.Quantity,  // Extract quantity from each order
    (a, b) => a + b,         // Sum the values
    seedValue: 0);           // Start with 0, use 0 for missing data

display("Aggregation definition created: Sum of quantities");

Error: (2,29): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(3,5): error CS0103: The name 'orders' does not exist in the current context

### Step 2: Define Customer Dimension (Rows)

A dimension definition specifies how to extract index values from source data and defines which values are relevant for analysis.

In [5]:
// Define customer dimension (rows)
var customerIdDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.CustomerId,  // Extract customer ID from orders
    title: "Customers",         // Dimension title for display
    IndexDefinition.Create("A", "Customer A"),
    IndexDefinition.Create("B", "Customer B"),
    IndexDefinition.Create("C", "Customer C"),
    IndexDefinition.Create("D", "Customer D"))
        .WithTrailingDefaultIndex("Total");  // Add total column at the end

display("Customer dimension created with 4 customers + Total");

Error: (2,27): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(3,5): error CS0103: The name 'orders' does not exist in the current context
(6,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(7,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(8,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(9,5): error CS0103: The name 'IndexDefinition' does not exist in the current context

### Step 3: Define Year Dimension (Columns)

We'll extract the year from the order date and create indexes for each year.

In [6]:
// Define year dimension (columns)
var yearDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.OrderDate.Year.ToString(),  // Extract year as string
    title: "Years",
    IndexDefinition.Create("2007", "2007 Year"),
    IndexDefinition.Create("2008", "2008 Year"),
    IndexDefinition.Create("2009", "2009 Year"))
        .WithTrailingDefaultIndex("Total");   // Add total row at the end

display("Year dimension created with 3 years + Total");

Error: (2,21): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(3,5): error CS0103: The name 'orders' does not exist in the current context
(6,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(7,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(8,5): error CS0103: The name 'IndexDefinition' does not exist in the current context

### Step 4: Build the Cube

Now we combine everything to build our data cube:

In [7]:
// Build the cube
var cube = orders.BuildCube(aggregationDefinition, customerIdDimension, yearDimension);

display("Data cube built successfully!");
display($"Cube dimensions: {cube.FreeDimensionCount}");
display($"Total value across all dimensions: {cube.GetValue()}");

Error: (2,12): error CS0103: The name 'orders' does not exist in the current context
(2,29): error CS0103: The name 'aggregationDefinition' does not exist in the current context
(2,52): error CS0103: The name 'customerIdDimension' does not exist in the current context
(2,73): error CS0103: The name 'yearDimension' does not exist in the current context

### Step 5: Query the Cube

Now let's explore different ways to query our cube:

In [8]:
// Basic queries
display("=== Basic Cube Queries ===");

// Total across all customers and years
var grandTotal = cube.GetValue();
display($"Grand Total: {grandTotal}");

// Total for Customer A across all years
var customerATotal = cube.GetValue("A");
display($"Customer A Total: {customerATotal}");

// Total for all customers in 2007
var year2007Total = cube.GetValue(default, "2007");
display($"2007 Total: {year2007Total}");

// Specific cell: Customer A in 2007
var specificValue = cube.GetValue("A", "2007");
display($"Customer A in 2007: {specificValue}");

// Non-existent data (returns seed value)
var unknownCustomer = cube.GetValue("Z", "2007");
display($"Unknown Customer Z in 2007: {unknownCustomer}");

Error: (5,18): error CS0103: The name 'cube' does not exist in the current context
(9,22): error CS0103: The name 'cube' does not exist in the current context
(13,21): error CS0103: The name 'cube' does not exist in the current context
(17,21): error CS0103: The name 'cube' does not exist in the current context
(21,23): error CS0103: The name 'cube' does not exist in the current context

### Step 6: Create a Complete Report

Let's transform our cube into a tabular report format:

In [9]:
display("=== Complete Report ===");
display("");

// Create report headers
var yearHeaders = yearDimension.Select(idx => idx.Title).ToArray();
var headerRow = $"{"Customers",-12} | {string.Join(" | ", yearHeaders.Select(h => $"{h,10}"))}";
var separatorRow = new string('-', 12 + 3 + yearHeaders.Length * 13);

display(headerRow);
display(separatorRow);

// Generate report rows
var reportRows = customerIdDimension.Select(customer =>
{
    var customerTitle = customer.Title.PadRight(12);
    var values = yearDimension.Select(year =>
    {
        var value = cube.GetValue(customer.Value, year.Value);
        return $"{value,10}";
    });

    return $"{customerTitle} | {string.Join(" | ", values)}";
});

display(reportRows);

Error: (5,19): error CS0103: The name 'yearDimension' does not exist in the current context
(13,18): error CS0103: The name 'customerIdDimension' does not exist in the current context
(16,18): error CS0103: The name 'yearDimension' does not exist in the current context
(18,21): error CS0103: The name 'cube' does not exist in the current context

## Chapter 2: Core Concepts Deep Dive

### Aggregation Definitions

Let's explore different types of aggregation patterns:

In [10]:
display("=== Different Aggregation Patterns ===");

// Sample numeric data
var numbers = new[] { 1, 5, 3, 9, 2, 7, 4, 8, 6 };

// 1. Sum aggregation
var sumAggregation = AggregationDefinition.Create(
    (int i) => i,
    (a, b) => a + b,
    0);
display("Sum aggregation created");

// 2. Count aggregation
var countAggregation = AggregationDefinition.Create(
    (int item) => 1,
    (a, b) => a + b,
    0);
display("Count aggregation created");

// 3. Minimum aggregation
var minAggregation = AggregationDefinition.Create(
    (int i) => i,
    (a, b) => Math.Min(a, b),
    int.MaxValue);
display("Minimum aggregation created");

// 4. Maximum aggregation
var maxAggregation = AggregationDefinition.Create(
    (int i) => i,
    (a, b) => Math.Max(a, b),
    int.MinValue);
display("Maximum aggregation created");

// 5. Collection aggregation
var collectionAggregation = AggregationDefinition.Create(
    (int i) => new[] { i },
    (a, b) => a.Concat(b).ToArray(),
    Array.Empty<int>());
display("Collection aggregation created");

Error: (7,22): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(14,24): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(21,22): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(28,22): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(35,29): error CS0103: The name 'AggregationDefinition' does not exist in the current context

### Complex Aggregations - Count and Sum Together

Sometimes you need multiple calculations per cell. Here's how to create composite aggregations:

In [11]:
// Define a composite type for count and sum
public struct CountAndSum
{
    public CountAndSum(int count, decimal sum)
    {
        Count = count;
        Sum = sum;
    }

    public int Count { get; }
    public decimal Sum { get; }
    public decimal Average => Count > 0 ? Sum / Count : 0;

    public static CountAndSum Zero => new(0, 0);

    public static CountAndSum Combine(CountAndSum left, CountAndSum right) =>
        new(left.Count + right.Count, left.Sum + right.Sum);

    public override string ToString() => $"Count: {Count}, Sum: {Sum}, Avg: {Average:F2}";
}

// Use composite aggregation with our orders
var compositeAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => new CountAndSum(1, order.Quantity),
    CountAndSum.Combine,
    CountAndSum.Zero);

var compositeCube = orders.BuildCube(compositeAggregation, customerIdDimension);

display("=== Composite Aggregation Results ===");
display($"Total across all customers: {compositeCube.GetValue()}");
display($"Customer A stats: {compositeCube.GetValue("A")}");
display($"Customer B stats: {compositeCube.GetValue("B")}");

Error: (23,28): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(24,5): error CS0103: The name 'orders' does not exist in the current context
(29,21): error CS0103: The name 'orders' does not exist in the current context
(29,60): error CS0103: The name 'customerIdDimension' does not exist in the current context

## Chapter 3: Advanced Dimension Features

### Hierarchical Indexes

Let's create dimensions with parent-child relationships for subtotals:

In [12]:
// Sample product data with categories
var products = new[] {
    new { ProductId = "P1", Name = "iPhone", Category = "Electronics", SubCategory = "Phones", Price = 999m },
    new { ProductId = "P2", Name = "Samsung Galaxy", Category = "Electronics", SubCategory = "Phones", Price = 899m },
    new { ProductId = "P3", Name = "iPad", Category = "Electronics", SubCategory = "Tablets", Price = 599m },
    new { ProductId = "P4", Name = "MacBook", Category = "Electronics", SubCategory = "Laptops", Price = 1299m },
    new { ProductId = "P5", Name = "T-Shirt", Category = "Clothing", SubCategory = "Shirts", Price = 25m },
    new { ProductId = "P6", Name = "Jeans", Category = "Clothing", SubCategory = "Pants", Price = 75m }
};

display("Sample product data:");
display(products.Select(product => $"  {product.Name} ({product.SubCategory}) - ${product.Price}"));

Sample product data:

[ iPhone (Phones) - $999, Samsung Galaxy (Phones) - $899, iPad (Tablets) - $599, MacBook (Laptops) - $1299, T-Shirt (Shirts) - $25, Jeans (Pants) - $75 ]

In [13]:
// Create hierarchical dimension with categories and subcategories
var hierarchicalDimension = DimensionDefinition.CreateForCollection(
    products,
    product => product.SubCategory,  // We map to subcategory
    title: "Product Categories",
    // Electronics category with subcategories
    IndexDefinition.Create(
        "Electronics", "Electronics Category",
        IndexDefinition.Create("Phones", "Mobile Phones"),
        IndexDefinition.Create("Tablets", "Tablets"),
        IndexDefinition.Create("Laptops", "Laptop Computers")),
    // Clothing category with subcategories
    IndexDefinition.Create(
        "Clothing", "Clothing Category",
        IndexDefinition.Create("Shirts", "Shirts"),
        IndexDefinition.Create("Pants", "Pants")))
    .WithTrailingDefaultIndex("All Products");

// Price aggregation
var priceAggregation = AggregationDefinition.CreateForCollection(
    products,
    product => product.Price,
    (a, b) => a + b,
    0m);

var hierarchicalCube = products.BuildCube(priceAggregation, hierarchicalDimension);

display("=== Hierarchical Dimension Results ===");
display($"Total value of all products: ${hierarchicalCube.GetValue():F2}");
display($"Electronics category total: ${hierarchicalCube.GetValue("Electronics"):F2}");
display($"Phones subcategory total: ${hierarchicalCube.GetValue("Phones"):F2}");
display($"Clothing category total: ${hierarchicalCube.GetValue("Clothing"):F2}");
display($"Shirts subcategory total: ${hierarchicalCube.GetValue("Shirts"):F2}");

Error: (2,29): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(7,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(9,9): error CS0103: The name 'IndexDefinition' does not exist in the current context
(10,9): error CS0103: The name 'IndexDefinition' does not exist in the current context
(11,9): error CS0103: The name 'IndexDefinition' does not exist in the current context
(13,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(15,9): error CS0103: The name 'IndexDefinition' does not exist in the current context
(16,9): error CS0103: The name 'IndexDefinition' does not exist in the current context
(20,24): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(26,33): error CS1061: '<anonymous type: string ProductId, string Name, string Category, string SubCategory, decimal Price>[]' does not contain a definition for 'BuildCube' and no accessible e

### Multi-selection Dimensions

Sometimes a single record can belong to multiple index values within the same dimension. For example, products with multiple tags:

In [14]:
// Sample data with multiple tags per order
var taggedOrders = new[] {
    new { OrderId = 1, Amount = 100m, Tags = new[] { "bestseller", "discount" } },
    new { OrderId = 2, Amount = 200m, Tags = new[] { "bestseller" } },
    new { OrderId = 3, Amount = 150m, Tags = new[] { "new", "discount" } },
    new { OrderId = 4, Amount = 75m, Tags = new[] { "new" } },
    new { OrderId = 5, Amount = 300m, Tags = new[] { "bestseller", "premium" } }
};

display("Tagged orders:");
display(taggedOrders.Select(order => $"  Order {order.OrderId}: ${order.Amount} - Tags: [{string.Join(", ", order.Tags)}]"));

Tagged orders:

[ Order 1: $100 - Tags: [bestseller, discount], Order 2: $200 - Tags: [bestseller], Order 3: $150 - Tags: [new, discount], Order 4: $75 - Tags: [new], Order 5: $300 - Tags: [bestseller, premium] ]

In [15]:
// Create multi-selection dimension
var tagsDimension = DimensionDefinition.CreateForCollectionWithMultiSelector(
    taggedOrders,
    order => order.Tags,              // Returns IEnumerable<string>
    title: "Tags",
    IndexDefinition.Create("bestseller", "Bestseller"),
    IndexDefinition.Create("discount", "Discounted"),
    IndexDefinition.Create("new", "New Products"),
    IndexDefinition.Create("premium", "Premium"))
    .WithTrailingDefaultIndex("Total");

var amountAggregation = AggregationDefinition.CreateForCollection(
    taggedOrders,
    order => order.Amount,
    (a, b) => a + b,
    0m);

var tagsCube = taggedOrders.BuildCube(amountAggregation, tagsDimension);

display("=== Multi-selection Dimension Results ===");
display($"Bestseller tag total: ${tagsCube.GetValue("bestseller"):F2}  (Orders 1, 2, 5: $100 + $200 + $300)");
display($"Discount tag total: ${tagsCube.GetValue("discount"):F2}    (Orders 1, 3: $100 + $150)");
display($"New tag total: ${tagsCube.GetValue("new"):F2}         (Orders 3, 4: $150 + $75)");
display($"Premium tag total: ${tagsCube.GetValue("premium"):F2}     (Order 5: $300)");
display($"Grand total: ${tagsCube.GetValue():F2}           (All unique orders: $825)");

Error: (2,21): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(6,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(7,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(8,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(9,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(12,25): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(18,29): error CS1061: '<anonymous type: int OrderId, decimal Amount, string[] Tags>[]' does not contain a definition for 'BuildCube' and no accessible extension method 'BuildCube' accepting a first argument of type '<anonymous type: int OrderId, decimal Amount, string[] Tags>[]' could be found (are you missing a using directive or an assembly reference?)

## Chapter 4: Advanced Querying

### Cube Slicing

Slicing allows you to fix certain dimensions and work with the remaining dimensions:

In [16]:
// Create a 3-dimensional cube: Customer x Year x Product
var detailedOrders = new[] {
    new { CustomerId = "A", Year = "2007", Product = "Laptop", Quantity = 10m },
    new { CustomerId = "A", Year = "2007", Product = "Mouse", Quantity = 5m },
    new { CustomerId = "A", Year = "2008", Product = "Laptop", Quantity = 15m },
    new { CustomerId = "B", Year = "2007", Product = "Laptop", Quantity = 8m },
    new { CustomerId = "B", Year = "2008", Product = "Mouse", Quantity = 12m },
    new { CustomerId = "B", Year = "2008", Product = "Keyboard", Quantity = 7m }
};

var detailedAggregation = AggregationDefinition.CreateForCollection(
    detailedOrders,
    order => order.Quantity,
    (a, b) => a + b,
    0m);

var customerDim = DimensionDefinition.CreateForCollection(
    detailedOrders,
    order => order.CustomerId,
    title: "Customers",
    IndexDefinition.Create("A", "Customer A"),
    IndexDefinition.Create("B", "Customer B"))
    .WithTrailingDefaultIndex("All Customers");

var yearDim = DimensionDefinition.CreateForCollection(
    detailedOrders,
    order => order.Year,
    title: "Years",
    IndexDefinition.Create("2007", "2007"),
    IndexDefinition.Create("2008", "2008"))
    .WithTrailingDefaultIndex("All Years");

var productDim = DimensionDefinition.CreateForCollection(
    detailedOrders,
    order => order.Product,
    title: "Products",
    IndexDefinition.Create("Laptop", "Laptop"),
    IndexDefinition.Create("Mouse", "Mouse"),
    IndexDefinition.Create("Keyboard", "Keyboard"))
    .WithTrailingDefaultIndex("All Products");

var threeDCube = detailedOrders.BuildCube(detailedAggregation, customerDim, yearDim, productDim);

display("3D Cube created: Customer x Year x Product");
display($"Total across all dimensions: {threeDCube.GetValue()}");

Error: (11,27): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(17,19): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(21,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(22,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(25,15): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(29,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(30,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(33,18): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(37,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(38,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(39,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(42

In [17]:
// Slice the cube to focus on Customer A's data
var customerASlice = threeDCube.Slice("A");

display("=== Customer A Slice (Year x Product) ===");
display($"Customer A total: {customerASlice.GetValue()}");
display($"Customer A 2007: {customerASlice.GetValue("2007")}");
display($"Customer A 2008: {customerASlice.GetValue("2008")}");
display($"Customer A Laptops: {customerASlice.GetValue(default, "Laptop")}");
display($"Customer A 2007 Laptops: {customerASlice.GetValue("2007", "Laptop")}");

// Slice by year
var year2007Slice = threeDCube.Slice(default, "2007");
display("");
display("=== 2007 Slice (Customer x Product) ===");
display($"2007 total: {year2007Slice.GetValue()}");
display($"2007 Customer A: {year2007Slice.GetValue("A")}");
display($"2007 Laptops: {year2007Slice.GetValue(default, "Laptop")}");

Error: (2,22): error CS0103: The name 'threeDCube' does not exist in the current context
(12,21): error CS0103: The name 'threeDCube' does not exist in the current context

### Breakdown Operations

Breakdown operations let you iterate through values along specific dimensions:

In [18]:
display("=== Breakdown by Customers ===");
var customerBreakdowns = threeDCube.BreakdownByDimensions(0) // First dimension (customers)
    .Select(customerBreakdown =>
    {
        var customerIndex = customerBreakdown.GetBoundIndexDefinition(0);
        var value = customerBreakdown.GetValue();
        return $"{customerIndex.Title}: {value}";
    });
display(customerBreakdowns);

display("");
display("=== Breakdown by Years ===");
var yearBreakdowns = threeDCube.BreakdownByDimensions(1) // Second dimension (years)
    .Select(yearBreakdown =>
    {
        var yearIndex = yearBreakdown.GetBoundIndexDefinition(1);
        var value = yearBreakdown.GetValue();
        return $"{yearIndex.Title}: {value}";
    });
display(yearBreakdowns);

display("");
display("=== Customer A Breakdown by Year and Product ===");
var customerAData = threeDCube.Slice("A");
foreach(var yearBreakdown in customerAData.BreakdownByDimensions(0)) // Years in sliced cube
{
    var yearIndex = yearBreakdown.GetBoundIndexDefinition(0);
    display($"\n{yearIndex.Title}:");

    var productDetails = yearBreakdown.BreakdownByDimensions(0) // Products
        .Where(productBreakdown => productBreakdown.GetValue() > 0) // Only show non-zero values
        .Select(productBreakdown =>
        {
            var productIndex = productBreakdown.GetBoundIndexDefinition(0);
            var value = productBreakdown.GetValue();
            return $"  {productIndex.Title}: {value}";
        });

    display(productDetails);
}

Error: (2,26): error CS0103: The name 'threeDCube' does not exist in the current context
(13,22): error CS0103: The name 'threeDCube' does not exist in the current context
(24,21): error CS0103: The name 'threeDCube' does not exist in the current context

## Chapter 5: Asynchronous Operations

CubeSharp supports building cubes from asynchronous data sources:

In [19]:
// Simulate an async data source
async IAsyncEnumerable<dynamic> GetOrdersAsync()
{
    var asyncOrders = new[] {
        new { CustomerId = "A", Year = "2007", Amount = 100m },
        new { CustomerId = "B", Year = "2007", Amount = 200m },
        new { CustomerId = "A", Year = "2008", Amount = 150m },
        new { CustomerId = "C", Year = "2008", Amount = 175m }
    };

    foreach(var order in asyncOrders)
    {
        // Simulate async delay
        await Task.Delay(100);
        yield return order;
    }
}

// Define aggregation and dimensions for async operation
var asyncAggregation = AggregationDefinition.Create(
    (dynamic order) => (decimal)order.Amount,
    (a, b) => a + b,
    0m);

var asyncCustomerDim = DimensionDefinition.Create(
    (dynamic order) => (string)order.CustomerId,
    title: "Customers",
    IndexDefinition.Create("A", "Customer A"),
    IndexDefinition.Create("B", "Customer B"),
    IndexDefinition.Create("C", "Customer C"))
    .WithTrailingDefaultIndex("Total");

var asyncYearDim = DimensionDefinition.Create(
    (dynamic order) => (string)order.Year,
    title: "Years",
    IndexDefinition.Create("2007", "2007"),
    IndexDefinition.Create("2008", "2008"))
    .WithTrailingDefaultIndex("Total");

display("Building cube from async data source...");
var asyncCube = await GetOrdersAsync().BuildCubeAsync(asyncAggregation, asyncCustomerDim, asyncYearDim);

display("=== Async Cube Results ===");
display($"Total: ${asyncCube.GetValue():F2}");
display($"Customer A: ${asyncCube.GetValue("A"):F2}");
display($"2007: ${asyncCube.GetValue(default, "2007"):F2}");
display($"Customer A 2007: ${asyncCube.GetValue("A", "2007"):F2}");

Error: (20,24): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(25,24): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(28,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(29,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(30,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(33,20): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(36,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(37,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(41,40): error CS1061: 'IAsyncEnumerable<dynamic>' does not contain a definition for 'BuildCubeAsync' and no accessible extension method 'BuildCubeAsync' accepting a first argument of type 'IAsyncEnumerable<dynamic>' could be found (are you missing a using directive or an

## Chapter 6: Real-world Example - Sales Dashboard

Let's create a comprehensive sales dashboard using all the concepts we've learned:

In [20]:
// Comprehensive sales data
var salesData = new[] {
    new { Date = new DateTime(2023, 1, 15), Region = "North", Category = "Electronics", Product = "Laptop", Quantity = 5, UnitPrice = 1200m, SalespersonId = "SP1" },
    new { Date = new DateTime(2023, 1, 20), Region = "North", Category = "Electronics", Product = "Phone", Quantity = 12, UnitPrice = 800m, SalespersonId = "SP1" },
    new { Date = new DateTime(2023, 2, 10), Region = "South", Category = "Electronics", Product = "Laptop", Quantity = 8, UnitPrice = 1200m, SalespersonId = "SP2" },
    new { Date = new DateTime(2023, 2, 15), Region = "North", Category = "Clothing", Product = "Jacket", Quantity = 20, UnitPrice = 150m, SalespersonId = "SP3" },
    new { Date = new DateTime(2023, 3, 5), Region = "South", Category = "Electronics", Product = "Tablet", Quantity = 15, UnitPrice = 600m, SalespersonId = "SP2" },
    new { Date = new DateTime(2023, 3, 12), Region = "East", Category = "Clothing", Product = "Shoes", Quantity = 25, UnitPrice = 80m, SalespersonId = "SP4" },
    new { Date = new DateTime(2023, 4, 8), Region = "East", Category = "Electronics", Product = "Phone", Quantity = 18, UnitPrice = 800m, SalespersonId = "SP4" },
    new { Date = new DateTime(2023, 4, 20), Region = "North", Category = "Clothing", Product = "Jacket", Quantity = 10, UnitPrice = 150m, SalespersonId = "SP3" }
};

display($"Sales data loaded: {salesData.Length} transactions");
display($"Total revenue: ${salesData.Sum(s => s.Quantity * s.UnitPrice):F2}");

Sales data loaded: 8 transactions

Total revenue: $55100.00

In [21]:
// Define comprehensive aggregations
var salesMetrics = AggregationDefinition.CreateForCollection(
    salesData,
    sale => new {
        Revenue = sale.Quantity * sale.UnitPrice,
        Quantity = sale.Quantity,
        TransactionCount = 1
    },
    (a, b) => new {
        Revenue = a.Revenue + b.Revenue,
        Quantity = a.Quantity + b.Quantity,
        TransactionCount = a.TransactionCount + b.TransactionCount
    },
    new { Revenue = 0m, Quantity = 0, TransactionCount = 0 });

// Define dimensions
var regionDimension = DimensionDefinition.CreateForCollection(
    salesData,
    sale => sale.Region,
    title: "Regions",
    IndexDefinition.Create("North", "Northern Region"),
    IndexDefinition.Create("South", "Southern Region"),
    IndexDefinition.Create("East", "Eastern Region"))
    .WithTrailingDefaultIndex("All Regions");

var categoryDimension = DimensionDefinition.CreateForCollection(
    salesData,
    sale => sale.Category,
    title: "Categories",
    IndexDefinition.Create("Electronics", "Electronics"),
    IndexDefinition.Create("Clothing", "Clothing"))
    .WithTrailingDefaultIndex("All Categories");

var monthDimension = DimensionDefinition.CreateForCollection(
    salesData,
    sale => sale.Date.ToString("yyyy-MM"),
    title: "Months",
    IndexDefinition.Create("2023-01", "January 2023"),
    IndexDefinition.Create("2023-02", "February 2023"),
    IndexDefinition.Create("2023-03", "March 2023"),
    IndexDefinition.Create("2023-04", "April 2023"))
    .WithTrailingDefaultIndex("All Months");

display("Dimensions defined: Region, Category, Month");

Error: (2,20): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(17,23): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(21,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(22,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(23,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(26,25): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(30,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(31,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(34,22): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(38,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(39,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(40,

In [22]:
// Build comprehensive sales cube
var salesCube = salesData.BuildCube(salesMetrics, regionDimension, categoryDimension, monthDimension);

display("=== Sales Dashboard - Overall Metrics ===");
var totalMetrics = salesCube.GetValue();
display($"Total Revenue: ${totalMetrics.Revenue:F2}");
display($"Total Units Sold: {totalMetrics.Quantity:N0}");
display($"Total Transactions: {totalMetrics.TransactionCount:N0}");
display($"Average Transaction Value: ${totalMetrics.Revenue / totalMetrics.TransactionCount:F2}");

display("");
display("=== Revenue by Region ===");
var regionBreakdowns = salesCube.BreakdownByDimensions(0)
    .Select(regionBreakdown =>
    {
        var region = regionBreakdown.GetBoundIndexDefinition(0);
        var metrics = regionBreakdown.GetValue();
        var percentage = totalMetrics.Revenue > 0 ? (metrics.Revenue / totalMetrics.Revenue * 100) : 0;
        return $"{region.Title,-16}: ${metrics.Revenue,10:F2} ({percentage,5:F1}%)";
    });
display(regionBreakdowns);

display("");
display("=== Revenue by Category ===");
var categoryBreakdowns = salesCube.BreakdownByDimensions(1)
    .Select(categoryBreakdown =>
    {
        var category = categoryBreakdown.GetBoundIndexDefinition(1);
        var metrics = categoryBreakdown.GetValue();
        var percentage = totalMetrics.Revenue > 0 ? (metrics.Revenue / totalMetrics.Revenue * 100) : 0;
        return $"{category.Title,-16}: ${metrics.Revenue,10:F2} ({percentage,5:F1}%)";
    });
display(categoryBreakdowns);

Error: (2,37): error CS0103: The name 'salesMetrics' does not exist in the current context
(2,51): error CS0103: The name 'regionDimension' does not exist in the current context
(2,68): error CS0103: The name 'categoryDimension' does not exist in the current context
(2,87): error CS0103: The name 'monthDimension' does not exist in the current context
(2,27): error CS1061: '<anonymous type: DateTime Date, string Region, string Category, string Product, int Quantity, decimal UnitPrice, string SalespersonId>[]' does not contain a definition for 'BuildCube' and no accessible extension method 'BuildCube' accepting a first argument of type '<anonymous type: DateTime Date, string Region, string Category, string Product, int Quantity, decimal UnitPrice, string SalespersonId>[]' could be found (are you missing a using directive or an assembly reference?)

In [23]:
// Monthly trend analysis
display("=== Monthly Revenue Trend ===");
var headerRow = $"{"Month",-15} {"Revenue",12} {"Quantity",10} {"Transactions",12} {"Avg/Trans",10}";
var separatorRow = new string('-', 65);
display(headerRow);
display(separatorRow);

var monthlyData = salesCube.BreakdownByDimensions(2)
    .Select(monthBreakdown =>
    {
        var month = monthBreakdown.GetBoundIndexDefinition(2);
        var metrics = monthBreakdown.GetValue();
        var avgTransaction = metrics.TransactionCount > 0 ? metrics.Revenue / metrics.TransactionCount : 0;

        if(metrics.Revenue > 0) // Skip months with no sales
        {
            return $"{month.Title,-15} ${metrics.Revenue,10:F2} {metrics.Quantity,9:N0} {metrics.TransactionCount,11:N0} ${avgTransaction,9:F2}";
        }
        return null;
    })
    .Where(row => row != null);

display(monthlyData);

// Regional breakdown by category
display("");
display("=== Regional Performance by Category ===");
foreach(var regionBreakdown in salesCube.BreakdownByDimensions(0))
{
    var region = regionBreakdown.GetBoundIndexDefinition(0);
    if(regionBreakdown.GetValue().Revenue == 0) continue; // Skip regions with no sales

    display($"\n{region.Title}:");

    var categoryDetails = regionBreakdown.BreakdownByDimensions(0) // Categories within region
        .Where(categoryBreakdown => categoryBreakdown.GetValue().Revenue > 0)
        .Select(categoryBreakdown =>
        {
            var category = categoryBreakdown.GetBoundIndexDefinition(0);
            var metrics = categoryBreakdown.GetValue();
            return $"  {category.Title,-12}: ${metrics.Revenue,8:F2} ({metrics.TransactionCount} transactions)";
        });

    display(categoryDetails);
}

Error: (8,19): error CS0103: The name 'salesCube' does not exist in the current context
(28,32): error CS0103: The name 'salesCube' does not exist in the current context

## Chapter 7: Best Practices and Performance Tips

### 1. Type Safety and Inference

Use type inference when possible, but be explicit when needed:

In [24]:
// ✅ Good: Use CreateForCollection for type inference
var goodAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => order.Quantity,
    (a, b) => a + b,
    0m);

// ✅ Good: Explicit types when inference fails
var explicitAggregation = AggregationDefinition.Create<dynamic, decimal>(
    order => order.Amount,
    (a, b) => a + b,
    0m);

display("Type safety examples completed");

Error: (2,23): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(3,5): error CS0103: The name 'orders' does not exist in the current context
(9,27): error CS0103: The name 'AggregationDefinition' does not exist in the current context

### 2. Memory Efficiency

Consider memory usage for large datasets:

In [25]:
// For large datasets, consider using structs for aggregation values
public struct EfficientMetrics
{
    public EfficientMetrics(decimal sum, int count)
    {
        Sum = sum;
        Count = count;
    }

    public decimal Sum { get; }
    public int Count { get; }
    public decimal Average => Count > 0 ? Sum / Count : 0;

    public static EfficientMetrics Combine(EfficientMetrics a, EfficientMetrics b) =>
        new(a.Sum + b.Sum, a.Count + b.Count);
}

// Use efficient aggregation
var efficientAggregation = AggregationDefinition.CreateForCollection(
    orders,
    order => new EfficientMetrics(order.Quantity, 1),
    EfficientMetrics.Combine,
    new EfficientMetrics(0, 0));

display("Memory-efficient aggregation pattern demonstrated");

Error: (19,28): error CS0103: The name 'AggregationDefinition' does not exist in the current context
(20,5): error CS0103: The name 'orders' does not exist in the current context

### 3. Error Handling

Handle edge cases gracefully:

In [26]:
// Safe division aggregation
public struct SafeRatio
{
    public SafeRatio(decimal numerator, decimal denominator)
    {
        Numerator = numerator;
        Denominator = denominator;
    }

    public decimal Numerator { get; }
    public decimal Denominator { get; }
    public decimal Ratio => Denominator != 0 ? Numerator / Denominator : 0;

    public static SafeRatio Combine(SafeRatio a, SafeRatio b) =>
        new(a.Numerator + b.Numerator, a.Denominator + b.Denominator);
}

// Handle null/missing data
var safeStringDimension = DimensionDefinition.CreateForCollection(
    orders,
    order => order.CustomerId ?? "Unknown",  // Handle null customer IDs
    title: "Customers",
    IndexDefinition.Create("A", "Customer A"),
    IndexDefinition.Create("Unknown", "Unknown Customers"))
    .WithTrailingDefaultIndex("Total");

display("Error handling patterns demonstrated");

Error: (19,27): error CS0103: The name 'DimensionDefinition' does not exist in the current context
(20,5): error CS0103: The name 'orders' does not exist in the current context
(23,5): error CS0103: The name 'IndexDefinition' does not exist in the current context
(24,5): error CS0103: The name 'IndexDefinition' does not exist in the current context

## Conclusion

Congratulations! You've completed the comprehensive CubeSharp tutorial. You've learned:

1. **Basic Concepts**: Aggregations, dimensions, and index definitions
2. **Advanced Features**: Hierarchical indexes, multi-selection dimensions, and default indexes
3. **Querying Techniques**: Value retrieval, slicing, and breakdown operations
4. **Asynchronous Operations**: Building cubes from async data sources
5. **Real-world Applications**: Complex reporting scenarios
6. **Best Practices**: Performance optimization and error handling

### Key Takeaways:

- **Systematic Approach**: CubeSharp provides a structured way to handle multi-dimensional data analysis
- **Flexibility**: Support for complex aggregations, hierarchies, and multi-selection scenarios
- **Performance**: In-memory processing with efficient querying capabilities
- **Maintainability**: Type-safe, testable code that's easy to refactor

### Next Steps:

- Experiment with your own data sources
- Create custom aggregation types for your specific needs
- Integrate with your existing .NET applications
- Explore the full API documentation for advanced scenarios

Happy coding with CubeSharp!

In [27]:
display("🎉 Tutorial completed successfully!");
display("Thank you for learning CubeSharp!");

🎉 Tutorial completed successfully!

Thank you for learning CubeSharp!